In [1]:
from sagas.conf.conf import cf
cf.data_dir

'/usr/local/saai/ai'

In [2]:
import allennlp.models

In [32]:
from allennlp.predictors.predictor import Predictor
# import allennlp_models.syntax.srl
predictor = Predictor.from_path(f"{cf.data_dir}/allenai/bert-base-srl-2020.03.24.tar.gz")

In [35]:
text='What time does he go to school'
predictor.predict(sentence=text)

{'verbs': [{'verb': 'go',
   'description': '[ARGM-TMP: What time] does [ARG0: he] [V: go] [ARG4: to school]',
   'tags': ['B-ARGM-TMP',
    'I-ARGM-TMP',
    'O',
    'B-ARG0',
    'B-V',
    'B-ARG4',
    'I-ARG4']}],
 'words': ['What', 'time', 'does', 'he', 'go', 'to', 'school']}

In [37]:
text='He goes to school at five a.m.'
predictor.predict(sentence=text)

{'verbs': [{'verb': 'goes',
   'description': '[ARG0: He] [V: goes] [ARG4: to school] [ARGM-TMP: at five a.m.]',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG4',
    'I-ARG4',
    'B-ARGM-TMP',
    'I-ARGM-TMP',
    'I-ARGM-TMP']}],
 'words': ['He', 'goes', 'to', 'school', 'at', 'five', 'a.m.']}

In [34]:
result=predictor.predict(
  sentence="The rent rose 10% from $3000 to $3300"
)
result

{'verbs': [{'verb': 'rose',
   'description': '[ARG1: The rent] [V: rose] [ARG2: 10 %] [ARG3: from $ 3000] [ARG4: to $ 3300]',
   'tags': ['B-ARG1',
    'I-ARG1',
    'B-V',
    'B-ARG2',
    'I-ARG2',
    'B-ARG3',
    'I-ARG3',
    'I-ARG3',
    'B-ARG4',
    'I-ARG4',
    'I-ARG4']}],
 'words': ['The',
  'rent',
  'rose',
  '10',
  '%',
  'from',
  '$',
  '3000',
  'to',
  '$',
  '3300']}

In [33]:
result=predictor.predict(
  sentence="Did Uriah honestly think he could beat the game in under three hours?"
)
result

{'verbs': [{'verb': 'think',
   'description': 'Did [ARG0: Uriah] [ARGM-MNR: honestly] [V: think] [ARG1: he could beat the game in under three hours] ?',
   'tags': ['O',
    'B-ARG0',
    'B-ARGM-MNR',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']},
  {'verb': 'could',
   'description': 'Did Uriah honestly think he [V: could] beat the game in under three hours ?',
   'tags': ['O',
    'O',
    'O',
    'O',
    'O',
    'B-V',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O']},
  {'verb': 'beat',
   'description': 'Did Uriah honestly think [ARG0: he] [ARGM-MOD: could] [V: beat] [ARG1: the game] in [ARGM-TMP: under three hours] ?',
   'tags': ['O',
    'O',
    'O',
    'O',
    'B-ARG0',
    'B-ARGM-MOD',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'O',
    'B-ARGM-TMP',
    'I-ARGM-TMP',
    'I-ARGM-TMP',
    'O']}],
 'words': ['Did',
  'Uriah',
  'honestly',
  'th

In [17]:
from sagas.nlu.utils import get_entities
print(result.keys(), [v['verb'] for v in result['verbs']])
for v in result['verbs']:
    print('-'*15)
    print(get_entities(v['tags']))
    print(v['description'])    

dict_keys(['verbs', 'words']) ['think', 'could', 'beat']
---------------
[('ARG0', 1, 1), ('ARGM-MNR', 2, 2), ('V', 3, 3), ('ARG1', 4, 12)]
Did [ARG0: Uriah] [ARGM-MNR: honestly] [V: think] [ARG1: he could beat the game in under three hours] ?
---------------
[('V', 5, 5)]
Did Uriah honestly think he [V: could] beat the game in under three hours ?
---------------
[('ARG0', 4, 4), ('ARGM-MOD', 5, 5), ('V', 6, 6), ('ARG1', 7, 8), ('ARGM-TMP', 10, 12)]
Did Uriah honestly think [ARG0: he] [ARGM-MOD: could] [V: beat] [ARG1: the game] in [ARGM-TMP: under three hours] ?


In [14]:
from sagas.nlu.utils import get_entities

def terms_positions(sents):
    terms=predictor.predict(sentence=sents)
    words = terms['words']
    rs = [] 
    for v in terms['verbs']:
        verb_terms=[]
        verb=v['verb']
        term_toks=get_entities(v['tags'])
        tokens=[(tag, ' '.join(words[start:end+1])) for tag,start,end in term_toks]
        # print(tokens)
        running_offset = 0        
        for tag, word in tokens:        
            word_offset = sents.index(word, running_offset)
            word_len = len(word)
            running_offset = word_offset + word_len
            verb_terms.append((word_offset, running_offset, verb+'_'+tag))
        rs.append(verb_terms)
    return rs

sentence="Did Uriah honestly think he could beat the game in under three hours?"
spans=terms_positions(sentence)
spans

[[(4, 9, 'think_ARG0'),
  (10, 18, 'think_ARGM-MNR'),
  (19, 24, 'think_V'),
  (25, 68, 'think_ARG1')],
 [(28, 33, 'could_V')],
 [(25, 27, 'beat_ARG0'),
  (28, 33, 'beat_ARGM-MOD'),
  (34, 38, 'beat_V'),
  (39, 47, 'beat_ARG1'),
  (51, 68, 'beat_ARGM-TMP')]]

In [15]:
from ipymarkup.palette import palette, BLUE, RED, GREEN
from ipymarkup import box_markup, show_box_markup
for s in spans:
    show_box_markup(sentence, s)

BoxMarkup('Did Uriah honestly think he could beat the game in under three hours?',
          [Span(4, 9, 'think_ARG0'),
           Span(10, 18, 'think_ARGM-MNR'),
           Span(19, 24, 'think_V'),
           Span(25, 68, 'think_ARG1')])

BoxMarkup('Did Uriah honestly think he could beat the game in under three hours?',
          [Span(28, 33, 'could_V')])

BoxMarkup('Did Uriah honestly think he could beat the game in under three hours?',
          [Span(25, 27, 'beat_ARG0'),
           Span(28, 33, 'beat_ARGM-MOD'),
           Span(34, 38, 'beat_V'),
           Span(39, 47, 'beat_ARG1'),
           Span(51, 68, 'beat_ARGM-TMP')])

In [38]:
terms_positions('"What sector is stepping forward to pick up the slack ? " he asked')

[[(1, 5, 'stepping_ARG1'),
  (6, 12, 'stepping_ARG0'),
  (16, 24, 'stepping_V'),
  (25, 32, 'stepping_ARGM-DIR'),
  (33, 53, 'stepping_ARGM-PRP')],
 [(36, 40, 'pick_V'), (44, 53, 'pick_ARG1')],
 [(1, 53, 'asked_ARG1'), (58, 60, 'asked_ARG0'), (61, 66, 'asked_V')]]

In [40]:
def show_spans(sentence):
    spans=terms_positions(sentence)
    for s in spans:
        show_box_markup(sentence, s)
show_spans('"What sector is stepping forward to pick up the slack ? " he asked')

BoxMarkup('"What sector is stepping forward to pick up the slack ? " he asked',
          [Span(1, 5, 'stepping_ARG1'),
           Span(6, 12, 'stepping_ARG0'),
           Span(16, 24, 'stepping_V'),
           Span(25, 32, 'stepping_ARGM-DIR'),
           Span(33, 53, 'stepping_ARGM-PRP')])

BoxMarkup('"What sector is stepping forward to pick up the slack ? " he asked',
          [Span(36, 40, 'pick_V'), Span(44, 53, 'pick_ARG1')])

BoxMarkup('"What sector is stepping forward to pick up the slack ? " he asked',
          [Span(1, 53, 'asked_ARG1'),
           Span(58, 60, 'asked_ARG0'),
           Span(61, 66, 'asked_V')])

The following types of modifiers are being used in PropBank:
* COM: Comitative 
* LOC: Locative 
* DIR: Directional 
* GOL: Goal
* MNR: Manner
* TMP: Temporal
* EXT: Extent
* REC: Reciprocals
* PRD: Secondary Predication 
* PRP: Purpose
* CAU: Cause
* DIS: Discourse 
* ADV: Adverbials 
* ADJ: Adjectival 
* MOD: Modal
* NEG: Negation 
* DSP: Direct Speech 
* LVB: Light Verb 
* CXN: Construction

In [5]:
predictor_openie = Predictor.from_path(f"{cf.data_dir}/allenai/openie-model.2020.03.26.tar.gz")
predictor_openie.predict(
  sentence="John decided to run for office next month."
)

{'verbs': [{'verb': 'decided',
   'description': '[ARG0: John] [V: decided] [ARG1: to run for office next month] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']},
  {'verb': 'run',
   'description': '[ARG0: John] decided to [V: run] [ARG1: for office] [ARGM-TMP: next month] .',
   'tags': ['B-ARG0',
    'O',
    'O',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARGM-TMP',
    'I-ARGM-TMP',
    'O']}],
 'words': ['John',
  'decided',
  'to',
  'run',
  'for',
  'office',
  'next',
  'month',
  '.']}

In [21]:
predictor_depp = Predictor.from_path(f"{cf.data_dir}/allenai/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")
r=predictor_depp.predict(
  sentence="If I bring 10 dollars tomorrow, can you buy me lunch?"
)
r

{'arc_loss': 0.37905213236808777,
 'tag_loss': 0.5370486974716187,
 'loss': 0.9161008596420288,
 'words': ['If',
  'I',
  'bring',
  '10',
  'dollars',
  'tomorrow',
  ',',
  'can',
  'you',
  'buy',
  'me',
  'lunch',
  '?'],
 'pos': ['SCONJ',
  'PRON',
  'VERB',
  'NUM',
  'NOUN',
  'NOUN',
  'PUNCT',
  'VERB',
  'PRON',
  'VERB',
  'PRON',
  'NOUN',
  'PUNCT'],
 'predicted_dependencies': ['mark',
  'nsubj',
  'advcl',
  'dep',
  'dobj',
  'tmod',
  'advmod',
  'aux',
  'nsubj',
  'root',
  'dobj',
  'dep',
  'discourse'],
 'predicted_heads': [3, 3, 10, 5, 3, 3, 10, 10, 10, 0, 10, 11, 10],
 'hierplane_tree': {'text': 'If I bring 10 dollars tomorrow , can you buy me lunch ?',
  'root': {'word': 'buy',
   'nodeType': 'root',
   'attributes': ['VERB'],
   'link': 'root',
   'spans': [{'start': 41, 'end': 45}],
   'children': [{'word': 'bring',
     'nodeType': 'advcl',
     'attributes': ['VERB'],
     'link': 'advcl',
     'spans': [{'start': 5, 'end': 11}],
     'children': [{'word': 

In [30]:
from anytree.importer import DictImporter
from anytree import RenderTree

sentence="Did Uriah honestly think he could beat the game in under three hours?"
r=predictor_depp.predict(sentence=sentence)
importer = DictImporter()
tree_root = importer.import_(r['hierplane_tree']['root'])
tree = RenderTree(tree_root)
for pre, fill, node in tree:
    print("%s%s: %s (%s)" % (pre, node.nodeType, node.word, 
                             ','.join(node.attributes).lower()))

root: think (verb)
├── nsubj: Uriah (propn)
├── dep: honestly (adv)
│   └── dep: Did (aux)
└── ccomp: beat (verb)
    ├── nsubj: he (pron)
    ├── aux: could (verb)
    ├── dobj: game (noun)
    │   └── det: the (det)
    ├── advmod: in (adp)
    │   └── dep: under (adp)
    │       └── dep: hours (noun)
    │           └── num: three (num)
    └── discourse: ? (punct)
